In [1]:
import numpy as np
from scipy.signal import fftconvolve
import superlet
import scipy.io
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class SuperletTransformCX:
    """
    Class used to compute the complex Superlet Transform of input data.
    """

    def __init__(   self,
                    inputSize,
                    samplingRate,
                    frequencyRange,
                    frequencyBins,
                    baseCycles,
                    superletOrders,
                    frequencies = None):
        """
        Initialize the superlet transform. 
        Arguments:
            inputSize: size of the input in samples
            samplingRate: the sampling rate of the input signal in Hz
            frequencyRange: tuplet of ascending frequency points, in Hz
            frequencyBins: number of frequency bins to sample in the interval frequencyRange
            baseCycles: number of cycles of the smallest wavelet (c1 in the paper)
            superletOrders: a tuple containing the range of superlet orders, linearly distributed along frequencyRange
            frequencies: specific list of frequencies - can be provided in stead of frequencyRange (it is ignored in this case)
        """
                # clear to reinit
        self.clear()

        # initialize containers
        if frequencies is not None:
            frequencyBins = len(frequencies)
            self.frequencies = frequencies
        else:
            self.frequencies = np.linspace(start=frequencyRange[0], stop=frequencyRange[1], num=frequencyBins)

        self.inputSize          = inputSize
        self.orders             = np.linspace(start=superletOrders[0], stop=superletOrders[1], num=frequencyBins)
        self.convBuffer         = np.zeros(inputSize, dtype=np.complex128)
        self.phaseBuffer        = np.zeros(inputSize, dtype=np.complex128)
        self.absBuffer          = np.zeros(inputSize, dtype=np.float64)
        self.magnitudeBuffer    = np.zeros(inputSize, dtype=np.float64)
        self.superlets          = []

        # create wavelets
        for iFreq in range(frequencyBins):
            centerFreq  = self.frequencies[iFreq]
            nWavelets   = int(np.ceil(self.orders[iFreq]))

            self.superlets.append([])
            for iWave in range(nWavelets):

                # create morlet wavelet
                self.superlets[iFreq].append(superlet.morlet(centerFreq, (iWave + 1) * baseCycles, samplingRate))



    def clear(self):
        """
        Clear the transform.
        """
        # fields
        self.inputSize          = None
        self.superlets          = None
        self.absBuffer          = None
        self.convBuffer         = None
        self.frequencies        = None
        self.orders             = None
        self.magnitudeBuffer    = None
        self.phaseBuffer        = None

    
    def transform(self, inputData):
        """
        Apply the superlet transform on a single data buffer.
        Arguments:
            inputData: A 1xInputSize array containing the signal to be transformed.
        """
        if len(inputData.shape) > 1 or inputData.shape[0] != self.inputSize:
            raise "Invalid input size."

        result = np.zeros((len(self.frequencies), self.inputSize), dtype=np.complex128)

        for iFreq in range(len(self.frequencies)):
            
            # init pooling buffer
            self.magnitudeBuffer.fill(1)
            self.phaseBuffer    .fill(0)

            if len(self.superlets[iFreq]) > 1:
                
                # superlet
                nWavelets   = int(np.floor(self.orders[iFreq]))
                rfactor     = 1.0 / nWavelets

                for iWave in range(nWavelets):
                    self.convBuffer         = fftconvolve(inputData, self.superlets[iFreq][iWave], "same")
                    self.absBuffer          = np.abs(self.convBuffer)
                    self.magnitudeBuffer    *= self.absBuffer
                    self.phaseBuffer        += self.convBuffer / self.absBuffer

                if superlet.fractional(self.orders[iFreq]) != 0 and len(self.superlets[iFreq]) == nWavelets + 1:

                    # apply the fractional wavelet
                    exponent    = self.orders[iFreq] - nWavelets
                    rfactor     = 1 / (nWavelets + exponent)

                    self.convBuffer         = fftconvolve(inputData, self.superlets[iFreq][nWavelets], "same")
                    self.absBuffer          = np.abs(self.convBuffer)
                    self.magnitudeBuffer    *= self.absBuffer ** exponent
                    self.phaseBuffer        += (self.convBuffer / self.absBuffer) * exponent

                # perform geometric mean
                result[iFreq, :] += (self.magnitudeBuffer ** rfactor) * (self.phaseBuffer / self.orders[iFreq])

            else:
                # wavelet transform
                result[iFreq, :] += fftconvolve(inputData, self.superlets[iFreq][0], "same")

        return result


# main superlet function
def superlets(data,
              fs,
              foi,
              c1,
              ord):
    """
    Perform fractional adaptive superlet transform (FASLT) on a single trial.
    Arguments:
        data: a numpy array of data
        fs: the sampling rate in Hz
        foi: list of frequencies of interest
        c1: base number of cycles parameter
        ord: the order (for SLT) or order range (for FASLT), spanned across the frequencies of interest
    Returns: a matrix containing the complex superlet spectrum
    """
    # determine buffer size
    bufferSize = data.shape[len(data.shape) - 1]

    # make order parameter
    if len(ord) == 1:
        ord = (ord, ord)

    # build the superlet analyzer
    faslt = SuperletTransformCX(inputSize        = bufferSize, 
                                frequencyRange   = None, 
                                frequencyBins    = None, 
                                samplingRate     = fs, 
                                frequencies      = foi, 
                                baseCycles       = c1, 
                                superletOrders   = ord)
        
    # apply transform
    result = faslt.transform(data)
    faslt.clear()

    return result

In [3]:
def draw_superlet(matrix,save_path,name,index):
    """
    input:
    martix: The calculation result of superlets is a complex matrix, with rows representing time and columns representing frequency.
    save_path: Address to save time-frequency maps.
    name: The file name of the mat file of the patient currently in use.
    index : The number of the currently processed channel, starting from 1, with a maximum of 148.
    name and index are mainly used to easily name the generated image.
    """
    amplitudes = np.abs(matrix)  # Extract amplitude information

    # Draw time-frequency maps.
    plt.figure(figsize=(10, 6))
    plt.imshow(amplitudes, aspect='auto', origin='lower', cmap='viridis')
    plt.xticks([]) #Remove the image ruler
    plt.yticks([])
    plt.axis('off')
    file_name = name +'_'+str(index)+ '.png'
    plt.savefig(f'{save_path}/{file_name}', bbox_inches='tight', pad_inches=0.0, dpi=150)
    plt.close()

def get_file_names(folder_path, file_type):
    files = os.listdir(folder_path)
    
    file_names = []
    
    for file in files:
        if file.endswith('.' + file_type):
            file_names.append(file)
            
    return file_names

In [ ]:
folder_path = '../儿科/2/'
file_type = 'mat'
result = get_file_names(folder_path, file_type)
for file_name in result:
    print(file_name)
    mat_data = scipy.io.loadmat(folder_path+file_name) 
    data = mat_data['Value']
    ez = np.array(mat_data['ez'][0])
    nez = np.array(mat_data['nez'][0])
    del mat_data
    for i in ez:
        save_path = './data2/1/'
        name = file_name[0:-4]
        index = i
        matrix = []
        for x in range(1,251):
            result = superlets(data[i-1], 500, [x], 7, [1,250])
            matrix.append(result[0])
        draw_superlet(matrix, save_path, name, index)
    for i in nez:
        save_path = './data2/0/'
        name = file_name[0:-4]
        index = i
        matrix = []
        for x in range(1,251):
            result = superlets(data[i-1], 500, [x], 7, [1,250])
            matrix.append(result[0])
        draw_superlet(matrix, save_path, name, index)

S001_EEG_SZ_02s.mat
S001_EEG_SZ_03s.mat
S001_EEG_SZ_04s.mat
S002_EEG_SZ_01s.mat
S002_EEG_SZ_02s.mat
S002_EEG_SZ_03s.mat
S005_EEG_SZa_02.mat
S008_EEG_SZs_05.mat
S008_EEG_SZs_02.mat
S008_EEG_SZa_04.mat
S008_EEG_SZa_03.mat
S008_EEG_SZa_01.mat
S006_EEG_SZs_02.mat
S006_EEG_SZs_01.mat
S005_EEG_SZs_01.mat
S001_EEG_SZ_01a.mat
